In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
# from langchain_core.vectorstores import FAISS
from pprint import pprint

# 1. Loader (웹문서)
from langchain_community.document_loaders import WebBaseLoader
from bs4.filter import SoupStrainer  # pip install beautifulsoup4

loader = WebBaseLoader(
    # 문서 출처 URL
    web_paths=('https://lilianweng.github.io/posts/2023-06-23-agent/', ),
    # 웹페이지 안에서 필요한 정보만 선택
    bs_kwargs={
        'parse_only': SoupStrainer(class_=['post-content']) 
    }
    # header_template={}
)
docs = loader.load()

# 2. Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_docs = splitter.split_documents(docs)
print(len(splitted_docs))

# 3. Embedding Model
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')  # small <-> large

# 4. Vectorstore (지금은 FAISS -> 클라우드-Pinecone)
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(splitted_docs, embedding=embedding)


In [11]:
from langgraph.graph import MessagesState, StateGraph
from langchain_core.tools import tool
builder = StateGraph(MessagesState) # 2개를 return 한다

@tool(response_format = 'content_and_artifact')
def retrieve(query:str):
    '''Retrieve information related to a query
    Args:
        query : Query to Search
    '''
    # 원본 Document list(artifact)
    docs = vectorstore.similarity_search(query, k = 3)
    # 편집한 텍스트 (content)
    result_text = '\n\n'.join(
        (f'Source:{doc.metadata}\nContent:{doc.page_content}')
        for doc in docs
    )
    return result_text, docs


retrieve('에이전트에 대해 설명해줘')

C:\Users\mingyu\AppData\Local\Temp\ipykernel_28020\2652278178.py:21: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieve('에이전트에 대해 설명해줘')


'Source:{\'source\': \'https://lilianweng.github.io/posts/2023-06-23-agent/\'}\nContent:}\n]\nThen after these clarification, the agent moved into the code writing mode with a different system message.\nSystem message:\n\nSource:{\'source\': \'https://lilianweng.github.io/posts/2023-06-23-agent/\'}\nContent:"content": "You will get instructions for code to write.\\nYou will write a very long answer. Make sure that every detail of the architecture is, in the end, implemented as code.\\nMake sure that every detail of the architecture is, in the end, implemented as code.\\n\\nThink step by step and reason yourself to the right decisions to make sure we get it right.\\nYou will first lay out the names of the core classes, functions, methods that will be necessary, as well as a quick comment on their purpose.\\n\\nThen you will output the content of each file including ALL code.\\nEach file must strictly follow a markdown code block format, where the following tokens must be replaced such t

In [26]:
from langchain_core.messages import SystemMessage
from langgraph.graph import MessagesState, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.graph import START, END


llm = ChatOpenAI(model='gpt-4.1',temperature = 0)

builder = StateGraph(MessagesState) # 'messages'

# Node
def query_or_respond(state:MessagesState):
    '''도구 호출을 하거나 최종 응답을 한다.'''
    llm_with_tools = llm.bind_tools([retrieve])
    res = llm_with_tools.invoke(state['messages'])
    return {'messages':[res]}

tools = ToolNode([retrieve])

  
def generate(state: MessagesState):
    """응답 생성"""
    tool_messages = []
    for msg in reversed(state['messages']): # 최신 메세지부터 순회
        if msg.type == 'tool':
            tool_messages.append(msg)
        else:
            break

    tool_messages.reverse()

    docs_content = '\n\n'.join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    # 필요 없는 Tool 메세지들을 제외하고, AI, Human, System 메세지만 모아서 정리
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}




In [27]:
# builder.add_node('query_or_respond',query_or_respond)
# builder.add_node(tools)
builder.add_node(generate)

builder.add_edge(START,'query_or_respond')
builder.add_conditional_edges(
    'query_or_respond',
    tools_condition,
    {END: END,'tools':'tools'} # 정확하게 상황별 다음 Node를 지정할 수 있음
    )
builder.add_edge('tools','generate')
builder.add_edge('generate',END)
graph = builder.compile()

ValueError: Found edge starting at unknown node 'query_or_respond'